In [1]:
import pandas as pd
import numpy as np
from random import sample
from datetime import datetime

In [2]:
path_name = "/home/user/Projects/owascrp/Outputs/Merged/"
file_name = "owa_merged_116_157.csv"

data_raw = pd.read_csv(path_name+file_name, delimiter=";")

/home/user/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_raw.head(2)

,Unnamed: 0,Auction,Auction_Close,Auction_No,By,Color,Condition,Date,Date_High,Date_Low,...,Publication,Size,Size_Cm,Size_Cm_H,Size_Cm_W,Size_Inches,Sold_For,Subject,Text,Title
0,0,116 - 9/20/2006,9/20/2006,116,Heinrich Bunting,hand color,B,1582,NaN,1582,...,,7 x 11.7 inches 17.8 x 29.8 cm,17.8x29.8,29.8,17.8,7x11.7,600,Title Pages,\n\nTitle page from Bunting's \nItinerarium Sa...,"Lot #1: ""Itinerarium Sacrae Scripturae Das ist..."
1,1,116 - 9/20/2006,9/20/2006,116,,hand color,,1595-1642,1642,1595,...,,0 x 0 inches 0.0 x 0.0 cm,0.0x0.0,0.0,0.0,0x0,140,Title Pages,\n\n1) This title page was for Johannes Blaeu...,"Lot #2: ""[Lot of 2]"" \n"


In [4]:
#data_raw[data_raw["Auction_No"]==153.00].head(2)

## Data Cleaning

### Eliminating Missing

In [5]:
 data_raw.rename(columns = {'Unnamed: 0':'Index'}, inplace = True)

In [6]:
print('---- In total %i rows ------' % len(data_raw))
for col in data_raw.columns:
    if sum(pd.isnull(data_raw[col]))>0:
      print("%i Null-Werte in Spalte %s" %(sum(pd.isnull(data_raw[col])), col))

---- In total 34571 rows ------
4 Null-Werte in Spalte Index
32501 Null-Werte in Spalte Date_High
3 Null-Werte in Spalte Date_Low
4 Null-Werte in Spalte Estimate
4 Null-Werte in Spalte Estimate_High
4 Null-Werte in Spalte Estimate_Low
4 Null-Werte in Spalte Image
4 Null-Werte in Spalte Lot
4 Null-Werte in Spalte Publication
8 Null-Werte in Spalte Size
8 Null-Werte in Spalte Size_Cm
8 Null-Werte in Spalte Size_Cm_H
8 Null-Werte in Spalte Size_Cm_W
8 Null-Werte in Spalte Size_Inches
8 Null-Werte in Spalte Sold_For
8 Null-Werte in Spalte Subject
8 Null-Werte in Spalte Text
8 Null-Werte in Spalte Title


In [7]:
#data_raw[data_raw["Size"].isnull()].head(2)

In [8]:
data_cleaner = data_raw[data_raw["Subject"].notnull()]
print('---- In total %i rows ------' % len(data_cleaner))

---- In total 34563 rows ------


In [9]:
data_cleaner = data_cleaner[data_cleaner["Date_Low"].notnull()]
print('---- In total %i rows ------' % len(data_cleaner))

---- In total 34560 rows ------


In [10]:
print('---- In total %i rows ------' % len(data_cleaner))
for col in data_cleaner.columns:
    if sum(pd.isnull(data_cleaner[col]))>0:
      print("%i Null-Werte in Spalte %s" %(sum(pd.isnull(data_cleaner[col])), col))

---- In total 34560 rows ------
32494 Null-Werte in Spalte Date_High


## Replacing wrong entries

In [11]:
#data_cleaner.columns

---- Correction of Auction Close Date ---- 

In [12]:
#np.unique(data_cleaner["Index"])
#np.unique(data_cleaner["Auction"])
#np.unique(data_cleaner["Auction_No"])
#np.unique(data_cleaner["Auction_Close"])

In [13]:
data_cleaner2 = data_cleaner
def make_date (x):
    return datetime.strptime(x, '%m/%d/%Y')
data_cleaner2["Auction_Close"]=data_cleaner2["Auction_Close"].apply(make_date)

In [14]:
#np.unique(data_cleaner["Auction_Close"])

----- Correction of Date_Low -----

In [15]:
#data_cleaner[data_cleaner["Date_Low"]=='`926']
#len(data_cleaner[data_cleaner["Date_Low"]!='`926'])
data_cleaner = data_cleaner[data_cleaner["Date_Low"]!='`926']

In [16]:
#data_cleaner[data_cleaner["Date_Low"] == 9999]

In [17]:
def left_4 (x):
    x = str(x)
    if len(x[:4])>3: 
        return int(x[:4])
    else:
        return int(9999)
data_cleaner["Date_Low"]=data_cleaner["Date_Low"].apply(left_4)
#np.unique(data_cleaner["Date_Low"])

/home/user/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
#len(data_cleaner[data_cleaner["Date_Low"]==9999])
data_cleaner = data_cleaner[data_cleaner["Date_Low"]!=9999]


---- Correction of Condition ---- 

In [19]:
np.unique(data_cleaner["Condition"])

array([' ', '     ', ' +A  ', ' +B  ', ' +C  ', ' A   ', ' A + ', ' A+  ',
       ' B   ', ' B+  ', ' C   ', ' C+  ', ' D   '], dtype=object)

In [20]:
#data_cleaner[data_cleaner["Condition"]== ' A + ']

In [21]:
#data_cleaner[data_cleaner["Condition"]== ' +A  '].head(2)

In [22]:
data_cleaner[data_cleaner["Index"] == 47][data_cleaner[data_cleaner["Index"] == 47]["Auction_No"]== 125]

,Index,Auction,Auction_Close,Auction_No,By,Color,Condition,Date,Date_High,Date_Low,...,Publication,Size,Size_Cm,Size_Cm_H,Size_Cm_W,Size_Inches,Sold_For,Subject,Text,Title
8176,47,125 - 9/24/2008,2008-09-24,125,Scherer,hand color,A +,1703,NaN,1703,...,Atlas Novus,14 x 9 inches 35.6 x 22.9 cm,35.6x22.9,22.9,35.6,14x9,1500,World - Polar,\n\nThis very graphic map of the world is pres...,"Lot #48: ""Repraesentatio Geographica Itineris ..."


In [23]:
def cond_repair_1(x):
    return x.replace(" ", "")
data_cleaner["Condition"]=data_cleaner["Condition"].apply(cond_repair_1)
np.unique(data_cleaner["Condition"])

array(['', '+A', '+B', '+C', 'A', 'A+', 'B', 'B+', 'C', 'C+', 'D'], dtype=object)

In [24]:
def cond_repair_2(x):
    if len(x)<1:
        x='None'
    else:
        if x[0]=='+':
            x=x[1]+x[0]
    return x
data_cleaner["Condition"]=data_cleaner["Condition"].apply(cond_repair_2)

In [25]:
#np.unique(data_cleaner["Condition"])

In [26]:
data_cleaner.columns

Index(['Index', 'Auction', 'Auction_Close', 'Auction_No', 'By', 'Color',
       'Condition', 'Date', 'Date_High', 'Date_Low', 'Estimate',
       'Estimate_High', 'Estimate_Low', 'Image', 'Lot', 'Publication', 'Size',
       'Size_Cm', 'Size_Cm_H', 'Size_Cm_W', 'Size_Inches', 'Sold_For',
       'Subject', 'Text', 'Title'],
      dtype='object')

--- Correction of Estimates and Sold_For -----

In [27]:
def to_int (x):
    return int(x)
data_cleaner["Estimate_High"] =data_cleaner["Estimate_High"].apply(to_int)

In [28]:
data_cleaner["Estimate_Low"] =data_cleaner["Estimate_Low"].apply(to_int)

In [29]:
data_cleaner["Sold_For"] =data_cleaner["Sold_For"].apply(to_int)


----- Correction of Cartographers (By) -----

In [30]:
def clean_carto(x):
    x = x.strip()
    if x == '' or x == '[blank]':
        x = 'Unknown'
    return x
data_cleaner["By"] = data_cleaner["By"].apply(clean_carto)
#data_cleaner[ data_cleaner["By"] == 'Unknown']

In [54]:
#len(np.unique(data_cleaner["By"]))

In [62]:
#np.unique(data_cleaner["By"])

----- Add Century and Sold binary -----

In [55]:
def data_century(x):
    x = str(x+100)
    x = x[0]+x[1]+'th'
    return x

def data_sold(y):
    z= 0
    if y > 0:
        z= 1
    return z

data_cleaner["Century"] = data_cleaner["Date_Low"].apply(data_century)
data_cleaner["Sold"] = data_cleaner["Sold_For"].apply(data_sold)

### Graphs and Analysis

In [56]:
data_cleaner.describe()

,Index,Auction_No,Date_Low,Estimate_High,Estimate_Low,Lot,Size_Cm_H,Size_Cm_W,Sold_For,Sold
count,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000,34557.000000
mean,428.050149,135.391527,1744.683827,715.282577,572.502098,429.050149,31.910174,38.321388,-2044.960963,0.760193
std,252.379831,11.993218,106.623170,1684.671711,1361.288388,252.379831,18.819407,23.494565,4573.601357,0.426972
min,0.000000,116.000000,1250.000000,15.000000,10.000000,1.000000,0.000000,0.000000,-10000.000000,0.000000
25%,210.000000,125.000000,1671.000000,200.000000,150.000000,211.000000,19.600000,22.900000,47.000000,1.000000
50%,424.000000,135.000000,1750.000000,325.000000,250.000000,425.000000,30.500000,36.100000,150.000000,1.000000
75%,640.000000,146.000000,1835.000000,600.000000,475.000000,641.000000,42.000000,50.800000,300.000000,1.000000
max,994.000000,157.000000,2010.000000,90000.000000,70000.000000,995.000000,335.300000,469.900000,52500.000000,1.000000


In [60]:
corr = data_cleaner.corr()
corr

,Index,Auction_No,Date_Low,Estimate_High,Estimate_Low,Lot,Size_Cm_H,Size_Cm_W,Sold_For,Sold
Index,1.000000,-0.086559,-0.209203,-0.124820,-0.127014,1.000000,-0.122373,-0.149250,-0.036715,-0.016498
Auction_No,-0.086559,1.000000,0.006634,0.035957,0.039631,-0.086559,0.008973,0.012661,-0.071437,-0.071778
Date_Low,-0.209203,0.006634,1.000000,-0.120918,-0.125733,-0.209203,0.237303,0.157628,0.009506,0.033829
Estimate_High,-0.124820,0.035957,-0.120918,1.000000,0.997351,-0.124820,0.148493,0.171261,0.139349,-0.028525
Estimate_Low,-0.127014,0.039631,-0.125733,0.997351,1.000000,-0.127014,0.146381,0.169827,0.137467,-0.029877
Lot,1.000000,-0.086559,-0.209203,-0.124820,-0.127014,1.000000,-0.122373,-0.149250,-0.036715,-0.016498
Size_Cm_H,-0.122373,0.008973,0.237303,0.148493,0.146381,-0.122373,1.000000,0.772977,0.012033,-0.012764
Size_Cm_W,-0.149250,0.012661,0.157628,0.171261,0.169827,-0.149250,0.772977,1.000000,0.011089,-0.020648
Sold_For,-0.036715,-0.071437,0.009506,0.139349,0.137467,-0.036715,0.012033,0.011089,1.000000,0.976920
Sold,-0.016498,-0.071778,0.033829,-0.028525,-0.029877,-0.016498,-0.012764,-0.020648,0.976920,1.000000


----- Analysing unsold lots -----

In [50]:
(data_cleaner["Sold_For"]==-10000).sum()

8287

In [57]:
data_cleaner[data_cleaner["Sold_For"]==-10000].head(1)

,Index,Auction,Auction_Close,Auction_No,By,Color,Condition,Date,Date_High,Date_Low,...,Size_Cm,Size_Cm_H,Size_Cm_W,Size_Inches,Sold_For,Subject,Text,Title,Century,Sold
8,8,116 - 9/20/2006,2006-09-20,116,Jacques Nicolas Bellin,uncolored,A,1772,NaN,1772,...,36.4x50.3,50.3,36.4,14.3x19.8,-10000,Title Pages,\n\nFine pair of French title pages from this ...,"Lot #9: ""[Lot of 2] Hydrographie Francoise Rec...",18th,0


----- Analysing Cartographers -----

In [79]:
data_cleaner['By'].head()

0         Heinrich Bunting
1                  Unknown
2         Abraham Ortelius
3         Mercator/Jansson
4    Johann Baptist Homann
Name: By, dtype: object

In [135]:
from collections import Counter
ct = Counter (list(data_cleaner['By']))
ctm = ct.most_common()

In [34]:
a = data_cleaner[data_cleaner["Sold_For"]>0]["Sold_For"]
b= data_cleaner[data_cleaner["Sold_For"]>0]["Date_Low"]
np.corrcoef(a, b)[0, 1]

-0.12361775374100703

### Modeling

In [108]:
data_cleaner.columns
#data_model = data_cleaner[['Auction_No', 'Date_Low']]
#data_model

Index(['Index', 'Auction', 'Auction_Close', 'Auction_No', 'By', 'Color',
       'Condition', 'Date', 'Date_High', 'Date_Low', 'Estimate',
       'Estimate_High', 'Estimate_Low', 'Image', 'Lot', 'Publication', 'Size',
       'Size_Cm', 'Size_Cm_H', 'Size_Cm_W', 'Size_Inches', 'Sold_For',
       'Subject', 'Text', 'Title', 'Century', 'Sold'],
      dtype='object')

In [128]:
data_model = data_cleaner [['Auction_No', 'Sold_For', 'By','Date_Low',  'Color', 'Condition', 'Size_Cm_H', 'Size_Cm_W', 'Subject']]
data_model = data_model[data_model['Sold_For']>0]



In [144]:
from sklearn import preprocessing
#Condition
le_cond = preprocessing.LabelEncoder()
le_cond.fit(data_model['Condition'])    
data_model['Condition'] = le_cond.transform(data_model['Condition']) 
#Subject
le_subj = preprocessing.LabelEncoder()
le_subj.fit(data_model['Subject'])    
data_model['Subject'] = le_subj.transform(data_model['Subject']) 
#Color
le_color = preprocessing.LabelEncoder()
le_color.fit(data_model['Color'])    
data_model['Color'] = le_color.transform(data_model['Color']) 
#By
le_by = preprocessing.LabelEncoder()
le_by.fit(data_model['By'])    
data_model['By'] = le_by.transform(data_model['By']) 



data_model.head(2)

,Auction_No,Sold_For,By,Date_Low,Color,Condition,Size_Cm_H,Size_Cm_W,Subject
0,116,600,664,1582,1,2,29.8,17.8,3128
1,116,140,1656,1595,1,7,0.0,0.0,3128


In [145]:
data_train = data_model[data_model["Auction_No"]<157]
data_test = data_model[data_model["Auction_No"]==157]

In [146]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=1)
model.fit(data_train.ix[:,2:], data_train.ix[:,1])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=10,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=1,
           verbose=0, warm_start=False)

In [158]:
from sklearn.metrics import mean_squared_error
from math import sqrt
# Make predictions.
predictions = model.predict(data_test.ix[:,2:])
# Compute the error.
sqrt(mean_squared_error(predictions, data_test.ix[:,1])) #RMSE

624.4765091814062

In [169]:
data_cleaner_sold = data_cleaner[data_cleaner['Sold_For']>0]
mean_est = ((data_cleaner_sold["Estimate_High"][data_cleaner_sold["Auction_No"]==157] + data_cleaner_sold["Estimate_Low"][data_cleaner_sold["Auction_No"]==157])/2)

In [171]:
comp_df = pd.DataFrame.from_items([('predictions', predictions), ('true_price', data_test.ix[:,1]), ('mean_est', mean_est)])

In [172]:
sqrt(mean_squared_error(comp_df['predictions'], comp_df['true_price'])) #RMSE

624.4765091814062

In [173]:
sqrt(mean_squared_error(comp_df['mean_est'], comp_df['true_price'])) #RMSE

667.9765038952324

In [176]:
comp_df = comp_df.sort(['true_price'], ascending=[1])
comp_df.describe()

/home/user/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,predictions,true_price,mean_est
count,574.000000,574.000000,574.000000
mean,519.058339,479.433798,719.303136
std,663.679470,980.856260,1546.345913
min,59.441306,11.000000,40.000000
25%,204.636527,100.000000,165.000000
50%,301.874375,200.000000,306.250000
75%,523.389870,450.000000,625.000000
max,5811.047787,11500.000000,17500.000000


In [199]:
from bokeh.plotting import figure, output_file, show

output_file("patch.html")

p = figure(plot_width=400, plot_height=400)

p.multi_line( [comp_df["predictions"], list(range(0, len(comp_df)))], [comp_df["true_price"],list(range(0, len(comp_df)))],
             color=["firebrick", "navy"], alpha=[0.8, 0.3], line_width=4)

show(p)




[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


### ALT

Fraction of Sales per Century

In [133]:
"""
data_to_graph = data_cleaner[["Auction_No", "Date_Low", "Sold_For"]]

def data_zero (a):
    if a > 0:
        return a
    else:
        return 0

data_to_graph["Cnt"] = 1
data_to_graph["Sold_For"] = data_to_graph["Sold_For"].apply(data_zero)
data_to_graph = data_to_graph[data_to_graph["Sold_For"] > 0].sort(['Auction_No', 'Date_Low'], ascending=[1, 0])
data_to_graph
"""

'\ndata_to_graph = data_cleaner[["Auction_No", "Date_Low", "Sold_For"]]\n\ndef data_zero (a):\n    if a > 0:\n        return a\n    else:\n        return 0\n\ndata_to_graph["Cnt"] = 1\ndata_to_graph["Sold_For"] = data_to_graph["Sold_For"].apply(data_zero)\ndata_to_graph = data_to_graph[data_to_graph["Sold_For"] > 0].sort([\'Auction_No\', \'Date_Low\'], ascending=[1, 0])\ndata_to_graph\n'

In [36]:
data_to_graph_group = data_to_graph.groupby(['Auction_No'], as_index=False)
data_to_graph_group_2 = data_to_graph_group.agg({"Sold" : np.mean, "Sold_For" : np.mean, "Cnt" : np.sum})
data_to_graph_group_2.head(2)

,Auction_No,Sold,Sold_For,Cnt
0,116,1,427.966278,771
1,117,1,422.672021,747


In [132]:
"""
data_to_graph_2 = data_cleaner[["Sold_For", "By"]]
data_to_graph_2 = data_to_graph_2[data_to_graph_2["Sold_For"] > 0]
data_to_graph_2 = data_to_graph_2[data_to_graph_2["By"] !='']
data_to_graph_group_3 = data_to_graph_2.groupby(['By'], as_index=False)
data_to_graph_group_3.size()
"""

'\ndata_to_graph_2 = data_cleaner[["Sold_For", "By"]]\ndata_to_graph_2 = data_to_graph_2[data_to_graph_2["Sold_For"] > 0]\ndata_to_graph_2 = data_to_graph_2[data_to_graph_2["By"] !=\'\']\ndata_to_graph_group_3 = data_to_graph_2.groupby([\'By\'], as_index=False)\ndata_to_graph_group_3.size()\n'

In [38]:
"""from bokeh.charts import Bar, output_file, show, hplot

#Bar Plots of sell rate and avg price
s1 = Bar(data_to_graph_group_2, label='Auction_No', values='Sold',  title="Sell Rate", ylabel="Sell Rate")
s2 = Bar(data_to_graph_group_2, label='Auction_No', values='Sold_For', title="Avg Price",color="navy", 
        ylabel="Avg Price")

output_file("test_owa_stacked_bar.html")

# put all the plots in an HBox
p = hplot(s1, s2)

# show the results
show(p)

"""




'from bokeh.charts import Bar, output_file, show, hplot\n\n#Bar Plots of sell rate and avg price\ns1 = Bar(data_to_graph_group_2, label=\'Auction_No\', values=\'Sold\',  title="Sell Rate", ylabel="Sell Rate")\ns2 = Bar(data_to_graph_group_2, label=\'Auction_No\', values=\'Sold_For\', title="Avg Price",color="navy", \n        ylabel="Avg Price")\n\noutput_file("test_owa_stacked_bar.html")\n\n# put all the plots in an HBox\np = hplot(s1, s2)\n\n# show the results\nshow(p)\n\n'